<a href="https://colab.research.google.com/github/chaw-thiri/DatabaseCourse/blob/main/its_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import sqlite3
import random
import pandas as pd

In [55]:
# Function to connect to the database
def connect_to_db(db_name):
    try:
        conn = sqlite3.connect(db_name)
        print(f"Connected to {db_name} successfully.")
        return conn
    except sqlite3.Error as e:
        print(f"Error occurred: {e}")
        return None

In [56]:
# Function to create tables in the database
def create_tables(conn):
    try:
        # creating FATL and STLVT tables first because they do not have foreign keys
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE "FATL" (
        "license" CHARACTER NOT NULL,
        expiration_date INTEGER NOT NULL,
        level INTEGER NOT NULL,
        PRIMARY KEY("license")
        );
        ''')

        cursor.execute('''
        CREATE TABLE "STLVT" (
        id INTEGER,
        level INTEGER NOT NULL,
        expiration_date INTEGER NOT NULL,
        authority TEXT NOT NULL,
        PRIMARY KEY(id)
        );
        ''')

        cursor.execute('''
        CREATE TABLE "booking service" (
        reference_id INTEGER NOT NULL,
        driver_id CHARACTER NOT NULL,
        driver_language CHARACTER NOT NULL,
        driver_location_id INTEGER NOT NULL,
        "game official_id" CHARACTER NOT NULL,
        "game official_language" CHARACTER NOT NULL,
        start_time DATETIME NOT NULL,
        end_time DATETIME NOT NULL,
        odometer INTEGER NOT NULL,
        location_id INTEGER NOT NULL,
        "vehicle_id" CHARACTER NOT NULL,
        "vehicle_avalilabe_time" DATETIME NOT NULL,
        "vehicle_status" BOOLEAN NOT NULL,
        PRIMARY KEY(reference_id),
        CONSTRAINT "driver_booking_service"
            FOREIGN KEY (driver_id, driver_language, driver_location_id)
            REFERENCES driver (id, language, location_id),
        CONSTRAINT "game_official_booking_service"
            FOREIGN KEY ("game official_id", "game official_language")
            REFERENCES "game official" (id, language),
        CONSTRAINT "location_booking_service"
            FOREIGN KEY (location_id) REFERENCES location (id),
        CONSTRAINT "vehicle_booking_service"
            FOREIGN KEY ("vehicle_id", "vehicle_avalilabe_time", "vehicle_status")
            REFERENCES "vehicle" (id, "avalilabe_time", status)
        );
        ''')

        cursor.execute('''
        CREATE TABLE country (
        name TEXT NOT NULL,
        language CHARACTER NOT NULL,
        PRIMARY KEY(name)
        );
        ''')

        cursor.execute('''
        CREATE TABLE driver (
        id CHARACTER NOT NULL,
        name TEXT NOT NULL,
        phone CHARACTER NOT NULL,
        driving_license TEXT NOT NULL,
        clearance_level INTEGER NOT NULL,
        "gender" CHARACTER NOT NULL,
        "FATL_license" CHARACTER,
        "STLVT_id" INTEGER,
        language CHARACTER NOT NULL,
        location_id INTEGER NOT NULL,
        PRIMARY KEY(id, language, location_id),
        CONSTRAINT "FATL_driver"
            FOREIGN KEY ("FATL_license") REFERENCES "FATL" ("license"),
        CONSTRAINT "STLVT_driver"
            FOREIGN KEY ("STLVT_id") REFERENCES "STLVT" (id),
        CONSTRAINT location_driver
            FOREIGN KEY (location_id) REFERENCES location (id)
        );
        ''')

        cursor.execute('''
        CREATE TABLE "game official" (
        id CHARACTER NOT NULL,
        name TEXT NOT NULL,
        role TEXT NOT NULL,
        language CHARACTER NOT NULL,
        "phone number" TEXT NOT NULL,
        gmail TEXT NOT NULL,
        kakao_id TEXT NOT NULL,
        country_name CHARACTER NOT NULL,
        city_id INTEGER NOT NULL,
        city_name CHARACTER NOT NULL,
        location_id INTEGER NOT NULL,
        PRIMARY KEY(id, language),
        CONSTRAINT "country_game_official"
            FOREIGN KEY (country_name) REFERENCES country (name),
        CONSTRAINT "location_game_official"
            FOREIGN KEY (location_id) REFERENCES location (id)
        );
        ''')

        cursor.execute('''
        CREATE TABLE location (
        id INTEGER NOT NULL,
        address TEXT NOT NULL,
        category TEXT NOT NULL,
        PRIMARY KEY(id)
        );
        ''')

        cursor.execute('''
        CREATE TABLE maintenance (
        "maintenance_id" CHARACTER NOT NULL,
        "vehicle_id" CHARACTER NOT NULL,
        category TEXT NOT NULL,
        odometer INTEGER NOT NULL,
        cost REAL NOT NULL,
        date DATE NOT NULL,
        "action" TEXT NOT NULL,
        "vehicle_avalilabe_time" DATETIME NOT NULL,
        "vehicle_status" BOOLEAN NOT NULL,
        PRIMARY KEY("maintenance_id", "vehicle_id"),
        CONSTRAINT "vehicle_maintenance"
            FOREIGN KEY ("vehicle_id", "vehicle_avalilabe_time", "vehicle_status")
            REFERENCES "vehicle" (id, "avalilabe_time", status)
        );
        ''')

        cursor.execute('''
        CREATE TABLE "vehicle" (
        id CHARACTER NOT NULL,
        "registration_id" CHARACTER NOT NULL,
        manufacturer TEXT NOT NULL,
        model TEXT NOT NULL,
        color TEXT NOT NULL,
        "current_odometer" INTEGER NOT NULL,
        "passenger_capacity" INTEGER NOT NULL,
        "avalilabe_time" DATETIME NOT NULL,
        location_id INTEGER NOT NULL,
        status BOOLEAN NOT NULL,
        PRIMARY KEY(id, "avalilabe_time", status),
        CONSTRAINT "location_vehicle"
            FOREIGN KEY (location_id) REFERENCES location (id)
        );
        ''')

    except sqlite3.Error as e:
        # Roll back the transaction in case of an error
        conn.rollback()
        print(f"Error occurred: {e}")

In [57]:
# Function to list all the tables in the database
def list_tables(conn):
    try:
        # Create a cursor object
        cursor = conn.cursor()

        # SQL query to fetch all tables
        cursor.execute('''
        SELECT name FROM sqlite_master WHERE type="table";
        ''')

        # Fetch all the tables
        tables = cursor.fetchall()

        # Display table names
        for table in tables:
            print(f"Table: {table[0]}")

    except sqlite3.Error as e:
        print(f"Error occurred: {e}")

In [59]:
# Function to insert sample data into tables
def insert_sample_data(conn):
    try:
        cursor = conn.cursor()

        # insert data into FATL
        cursor.execute("INSERT INTO 'FATL' ('license', expiration_date, level) VALUES ('ABCD123', 20231231, 1), ('EFGH456', 20230630, 2), ('IJKL789', 20240915, 3), ('MNOP123', 20250705, 2), ('QRST567', 20261120, 1);")

        # insert data into SALVT
        STLVT_data = [
            (1, 1, 20231231, "Local Police"),
            (2, 2, 20230630, "Local Police"),
            (3, 3, 20240915, "Foreign Police"),
            (4, 2, 20250705, "Foreign Police"),
            (5, 1, 20261120, "Foreign Police")
        ]
        cursor.executemany('INSERT INTO "STLVT" (id, level, expiration_date, authority) VALUES (?, ?, ?, ?)', STLVT_data)

        # insert data into location
        location_set = ['Hotel', 'Airport', 'Conference', 'Apartment']
        location_data = [
            (1, '123 Lotte St', random.choice(location_set)),
            (2, '456 Inha St', random.choice(location_set)),
            (3, '789 Minchuhol St', random.choice(location_set)),
            (4, '101 Songdo St', random.choice(location_set)),
            (5, '202 Gangnum St', random.choice(location_set))
        ]
        cursor.executemany('INSERT INTO location (id, address, category) VALUES (?, ?, ?)', location_data)

        # insert data into vehicle
        vehicle_data = [
            ("V1000", "2001 ABC", "Volvo", "XC90 SE", "Silver", 4350, 4, '2023-10-21 08:00:00', 1, 1),
            ("V1001", "2006 AFD", "Kia", "K7", "Black", 2195, 4, '2023-10-21 09:00:00', 2, 1),
            ("V1002", "2021 AHR", "Tesla", "2020 F", "White", 509, 4, '2023-10-21 10:00:00', 3, 1),
            ("V1003", "2020 DXF", "Ford", "Transit", "Silver", 974, 2, '2023-10-21 11:00:00', 4, 0),
            ("V1004", "2022 XYZ", "Hyundai", "R2D2", "Black", 999, 10, '2023-10-21 12:00:00', 5, 1),
        ]
        cursor.executemany('''
            INSERT INTO "vehicle" (id, "registration_id", manufacturer, model, color, "current_odometer",
                "passenger_capacity", "avalilabe_time", location_id, status)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', vehicle_data)

        # insert data into maintenence
        maintenance_data = [
            ("M1000", "V1000", "maintenance", 5000, 50.0, '2023-10-21', "Performed oil change", '2023-10-21 13:00:00', 1),
            ("M1001", "V1001", "maintenance", 3000, 30.0, '2023-10-20', "Rotated tires", '2023-10-20 14:00:00', 1),
            ("M1002", "V1002", "maintenance", 1000, 20.0, '2023-10-19', "Inspected brakes", '2023-10-19 15:00:00', 1),
            ("M1003", "V1003", "repair", 15000, 100.0, '2023-10-18', "Serviced transmission", '2023-10-18 16:00:00', 0),
            ("M1004", "V1004", "maintenance", 2000, 80.0, '2023-10-17', "Replaced battery", '2023-10-17 17:00:00', 1),
        ]
        cursor.executemany('''
            INSERT INTO "maintenance" ("maintenance_id", "vehicle_id", category, odometer, cost, date, "action", "vehicle_avalilabe_time", "vehicle_status")
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', maintenance_data)

        # insert data into driver table
        driver_data = [
            ("D1000", "Lee Min Ho", "123-456-7890", "A12345", 1, "Male", "F1000", 1, "KOR", 1),
            ("D1001", "Nabora", "987-654-3210", "B67890", 2, "Female", "F1001", 2, "JPN", 2),
            ("D1002", "Wang Yi Bo", "555-555-5555", "C54321", 3, "Male", "F1002", 3, "CHN", 3),
            ("D1003", "John Smith", "111-222-3333", "D99999", 4, "Male", "F1003", 4, "ENG", 4),
            ("D1004", "Shwe Yi", "666-777-8888", "E77777", 5, "Female", "F1004", 5, "BUR", 5),
        ]
        cursor.executemany('''
            INSERT INTO "driver" (id, name, phone, driving_license, clearance_level, "gender", "FATL_license", "STLVT_id", language, location_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', driver_data)

        # insert data into game official
        game_official_data = [
            ("GO1000", "John Doe", "coach", "KR", "123-456-7890", "john.doe@gmail.com", "kakao123", "Korea", 1, "Seoul", 1),
            ("GO1001", "Jane Smith", "judge", "JP", "987-654-3210", "jane.smith@gmail.com", "kakao456", "Japan", 2, "Busan", 2),
            ("GO1002", "Dr. Lee", "med aid", "CN", "555-555-5555", "dr.lee@gmail.com", "kakao789", "China", 3, "Incheon", 3),
            ("GO1003", "Mr. Kim", "authority", "GB", "111-222-3333", "mr.kim@gmail.com", "kakaoabc", "England", 4, "Daegu", 4),
            ("GO1004", "Lucas Wong", "doctor", "EN", "666-777-8888", "lucas.wong@gmail.com", "kakaoxyz", "US", 5, "Ulsan", 5),
        ]
        cursor.executemany('''
            INSERT INTO "game official" (id, name, role, language, "phone number", gmail, kakao_id, country_name, city_id, city_name, location_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', game_official_data)

        # insert data into country
        country_data = [
            ("Korea", "KR"),
            ("Japan", "JP"),
            ("China", "CN"),
            ("England", "GB"),
            ("US", "EN"),
        ]
        cursor.executemany('INSERT INTO country (name, language) VALUES (?, ?)', country_data)

        # insert data into booking table
        booking_service_data = [
            (1, "D1000", "KR", 1, "GO1000", "KR", '2023-10-21 14:00:00', '2023-10-21 16:00:00', 5000, 1, "V1000", '2023-10-21 13:00:00', 1),
            (2, "D1001", "JP", 2, "GO1001", "JP", '2023-10-22 15:00:00', '2023-10-22 17:00:00', 3000, 2, "V1001", '2023-10-22 14:00:00', 1),
            (3, "D1002", "CN", 3, "GO1002", "CN", '2023-10-23 16:00:00', '2023-10-23 18:00:00', 1000, 3, "V1002", '2023-10-23 15:00:00', 1),
            (4, "D1003", "GB", 4, "GO1003", "GB", '2023-10-24 17:00:00', '2023-10-24 19:00:00', 15000, 4, "V1003", '2023-10-24 16:00:00', 0),
            (5, "D1004", "EN", 5, "GO1004", "EN", '2023-10-25 18:00:00', '2023-10-25 20:00:00', 2000, 5, "V1004", '2023-10-25 17:00:00', 1),
        ]
        cursor.executemany('''
            INSERT INTO "booking service" (reference_id, driver_id, driver_language, driver_location_id, "game official_id", "game official_language",
                                            start_time, end_time, odometer, location_id, "vehicle_id", "vehicle_avalilabe_time", "vehicle_status")
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', booking_service_data)


        conn.commit()

    except sqlite3.Error as e:
        conn.rollback()
        print(f"Error occurred: {e}")

In [60]:
# Function to show rows of all tables
def show_rows(conn):
    try:
        cursor = conn.cursor()
        # List of table names
        tables = ['FATL', 'STLVT', 'booking service', 'country', 'driver', 'game official', 'location', 'maintenance', 'vehicle']
        # Fetch and display rows for each table
        for table in tables:
            print(f"\nRows from {table} table:")
            cursor.execute(f'SELECT * FROM "{table}"')
            rows = cursor.fetchall()
            for row in rows:
                print(row)

    except sqlite3.Error as e:
        print(f"Error occurred: {e}")

In [63]:
if __name__ == "__main__":
    db_name = 'its_company.db'
    conn = connect_to_db(db_name)
    create_tables(conn)
    list_tables(conn)
    conn.close()


Connected to its_company.db successfully.
Table: FATL
Table: STLVT
Table: booking service
Table: country
Table: driver
Table: game official
Table: location
Table: maintenance
Table: vehicle


In [65]:

if __name__ == "__main__":

    db_name = 'its_company.db'
    conn = connect_to_db(db_name)

    insert_sample_data(conn)
    show_rows(conn)
    conn.close()



Connected to its_company.db successfully.

Rows from FATL table:
('ABCD123', 20231231, 1)
('EFGH456', 20230630, 2)
('IJKL789', 20240915, 3)
('MNOP123', 20250705, 2)
('QRST567', 20261120, 1)

Rows from STLVT table:
(1, 1, 20231231, 'Local Police')
(2, 2, 20230630, 'Local Police')
(3, 3, 20240915, 'Foreign Police')
(4, 2, 20250705, 'Foreign Police')
(5, 1, 20261120, 'Foreign Police')

Rows from booking service table:
(1, 'D1000', 'KR', 1, 'GO1000', 'KR', '2023-10-21 14:00:00', '2023-10-21 16:00:00', 5000, 1, 'V1000', '2023-10-21 13:00:00', 1)
(2, 'D1001', 'JP', 2, 'GO1001', 'JP', '2023-10-22 15:00:00', '2023-10-22 17:00:00', 3000, 2, 'V1001', '2023-10-22 14:00:00', 1)
(3, 'D1002', 'CN', 3, 'GO1002', 'CN', '2023-10-23 16:00:00', '2023-10-23 18:00:00', 1000, 3, 'V1002', '2023-10-23 15:00:00', 1)
(4, 'D1003', 'GB', 4, 'GO1003', 'GB', '2023-10-24 17:00:00', '2023-10-24 19:00:00', 15000, 4, 'V1003', '2023-10-24 16:00:00', 0)
(5, 'D1004', 'EN', 5, 'GO1004', 'EN', '2023-10-25 18:00:00', '2023-1